<a href="https://colab.research.google.com/github/IbramMedhat/Human-Motion-Synthesis/blob/main/simple_keras_implementation_with_mpjpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.8
import numpy as np
from tensorflow.keras import Model as Model_
from tensorflow.keras.layers import Input, ReLU, LSTM, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pandas as pd


# torch.manual_seed(0) # Set for testing purposes, please do not change!

print(tf.keras.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.8.0


In [2]:
!pip install tensorflow==2.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 668.3 MB 17 kB/s 
     |████████████████████████████████| 462 kB 31.0 MB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
    Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707


In [3]:
#!pip install tensorflow_model_remediation

In [4]:
# #Need only to be used with google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import re

class Dataset_Preprocessing:
    def __init__(self, dir_path, include_dimension = 2, sample_size = 50, total_classes = 17):
        
        #Dataset Directory path
        self.dir_path = dir_path
        
        #Which Dimension file to include, possible values: 2 and 3
        self.include_dimension = include_dimension
        
        #Total frames in one Sample
        self.sample_size = sample_size
        
        #Activity classes to include
        self.classes = ['SittingDown', 'Walking', 'Directions', 'Discussion', 'Sitting', 'Phoning', 'Eating', 'Posing', 'Greeting', 'Smoking']
        
        #Total activity classes
        self.total_classes = len(self.classes)
        
        #Subject Folders names in the Dataset
        self.internal_folders = ['S1', 'S5','S6','S7','S8','S9','S11']
    
    def read_dataset(self):
        try:
            #Contains all the different activity vectors
            activity_vector = {}
            
            #Contains the overall dataset
            sampled_data = None
            
            #Based on dimensions, which folder to use for extracting the dataset files
            data_folder = 'Poses_D2_Positions' if self.include_dimension == 2 else 'Poses_D3_Positions'
            
            #Checking if the dataset path is valid
            if not os.path.exists(self.dir_path):
                print('The Data Directory Does not Exist!')
                return None

            #Iterating over all the subject folders
            for fld in self.internal_folders:
                #Iterating for each file in the specified folder
                for file in os.listdir(os.path.join(self.dir_path, fld, data_folder)):
                    #Extracting the activity from the filename
                    activity = self.__extract_activity(file)
                    
                    if activity not in self.classes:
                        continue
                    
                    #Reading the CSV file using Pandas
                    data = pd.read_csv(os.path.join(self.dir_path, fld, data_folder, file), header=None)

                    #Formulating the activity vector using one hot encoding
                    if activity not in activity_vector:
                        total_keys = len(activity_vector.keys())
                        activity_vector[activity] = np.zeros(self.total_classes)
                        activity_vector[activity][total_keys] = 1
                    vector = activity_vector[activity]
                    
                    #Sampling the dataset
                    grouped_sample = self.__group_samples(data, self.sample_size, vector)
                    sampled_data = grouped_sample if sampled_data is None else np.append(sampled_data, grouped_sample, axis=0)
                    
            return sampled_data
        except Exception as e:
            print(e)
    
    def __extract_activity(self, filename):
        try:
            #Extracting the filename and excluding the extension
            name = os.path.splitext(filename)[0]
            
            #Substituting the empty string with characters other than english alphabets
            activity = re.sub('[^A-Za-z]+' , '' , name)
            return activity
        except Exception as e:
            print(e)
    
    def __group_samples(self, dataset, sample_size, activity):
        try:
            #Checking if the dataset is a Pandas Dataframe
            if not isinstance(dataset, pd.DataFrame):
                print('Expecting Pandas Dataframe, but got {}'.format(type(dataset)))
                return None
            
            #Appending activity class to each row in the dataset
            dataset = pd.concat([dataset, pd.DataFrame(np.tile(activity, (dataset.shape[0],1)))], axis=1)
            
            #Reshaping the dataset into sample batches
            total_samples = dataset.shape[0]//sample_size
            total_features = dataset.shape[1]
            grouped_rows = dataset.to_numpy()[:total_samples*self.sample_size].reshape((-1,self.sample_size, total_features))
            
            return grouped_rows
        except Exception as e:
            print(e)

In [6]:
#For short term prediction, we need a sample size of 20(10 frames input sequance, 10 frames predicted sequance)
sampled_data = Dataset_Preprocessing('/content/drive/MyDrive/Colab Notebooks/H3.6csv', sample_size=20).read_dataset()

In [7]:
def split_to_features_labels(dataset, input_sequance_size=10) :
    assert input_sequance_size < dataset.shape[1], f"input sequance should be smaller than the total sample size"
    features = dataset[:, np.s_[0:input_sequance_size], :]
    labels = dataset[:,np.s_[input_sequance_size:], :64]
    
    return features, labels

In [8]:
sampled_dataX, sampled_dataY = split_to_features_labels(sampled_data, input_sequance_size=10)


In [9]:
print('Total Samples: {}'.format(sampled_dataY.shape[0]))
print('Total Frames: {}'.format(sampled_dataY.shape[1]))
print('Total Features: {}'.format(sampled_dataY.shape[2]))

Total Samples: 77144
Total Frames: 10
Total Features: 64


In [10]:
class GloGen(Model_):
    def __init__(self, enocder_hidden_state=200, decoder_hidden_state=200, output_diminsion=64, activation='relu'):
        super(GloGen, self).__init__()
        self.encoder = LSTM(enocder_hidden_state, return_state=True, return_sequences=True)
        self.decoder = LSTM(decoder_hidden_state, return_sequences=True, return_state=True)
        self.dense_layer = TimeDistributed(Dense(output_diminsion, activation=activation)) 

    def call(self, inputs):
        encoder_outputs, state_h, state_c = self.encoder(inputs)
        encoder_states = [state_h, state_c]
        output, _, _ = self.decoder(encoder_outputs, initial_state=encoder_states)
        output = self.dense_layer(output)
        return output

In [11]:
# Define the model that will turn
# `encoder_input_data` into `decoder_target_data`
glogen_model = GloGen()

In [12]:
class JointLoss() :
  def __init__(self, lambda1=0.5, lambda2=0.5) :
    self.lambda1 = lambda1
    self.lambda2 = lambda2

  def loss_joint(self, predicted_sequance_batch, target_sequance_batch) :
      diff_norm_2 = tf.math.reduce_sum(tf.square(tf.subtract(predicted_sequance_batch, target_sequance_batch)), axis=2)
      return tf.reduce_sum(diff_norm_2, axis=1) 

  def loss_motion_flow(self, predicted_sequance_batch, target_sequance_batch) :
      predictions_tomporal_diffs = tf.experimental.numpy.diff(predicted_sequance_batch, axis=1)
      real_tomporal_diffs = tf.experimental.numpy.diff(target_sequance_batch, axis=1)
      prediction_motion_flow_diff_norm_2 = tf.reduce_sum(tf.square(tf.subtract(predictions_tomporal_diffs, real_tomporal_diffs)), axis=2)
      return tf.reduce_sum(prediction_motion_flow_diff_norm_2, axis=1)


  def total_loss(self, target_sequance_batch, predicted_sequance_batch) :
      joints_loss = self.loss_joint(predicted_sequance_batch, target_sequance_batch)
      motion_flow_loss = self.loss_motion_flow(predicted_sequance_batch, target_sequance_batch)
      return self.lambda1*joints_loss + self.lambda2*motion_flow_loss



In [13]:
def run_experiment(learning_rate=0.002, lambda1=0.5, lambda2=0.5, metrics=None, batch_size=100, epochs=50, validation_split=0.2) :
  glogen_model = GloGen()
  loss_function = JointLoss(lambda1=lambda1, lambda2=lambda2).total_loss
  glogen_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                       loss=loss_function, metrics=metrics,run_eagerly=True)
  history = glogen_model.fit(sampled_dataX, sampled_dataY,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=validation_split)
  return history

In [14]:
#history = run_experiment(epochs=10, lambda1=0.8, lambda2=0.2, metrics=['mse'])

In [15]:
def compute_kernel(predicted_sequance_batch, target_sequance_batch):
    x_size = tf.shape(predicted_sequance_batch)[0]
    y_size = tf.shape(target_sequance_batch)[0]
    dim = tf.shape(predicted_sequance_batch)[1]
    tiled_x = tf.tile(tf.reshape(predicted_sequance_batch, tf.stack([x_size, 1, dim])), tf.stack([1, y_size, 1]))
    tiled_y = tf.tile(tf.reshape(target_sequance_batch, tf.stack([1, y_size, dim])), tf.stack([x_size, 1, 1]))
    return tf.exp(-tf.reduce_mean(tf.square(tiled_x - tiled_y), axis=2) / tf.cast(dim, tf.float32))


def MMD(predicted_sequance_batch, target_sequance_batch):
    x_kernel = compute_kernel(predicted_sequance_batch, predicted_sequance_batch)
    y_kernel = compute_kernel(target_sequance_batch, target_sequance_batch)
    xy_kernel = compute_kernel(predicted_sequance_batch, target_sequance_batch)
    return tf.reduce_mean(x_kernel) + tf.reduce_mean(y_kernel) - 2 * tf.reduce_mean(xy_kernel)

In [16]:
from keras import backend as K
def MPJPE(y_true, y_pred):
    return K.mean(K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1)))

In [17]:
from keras import backend as K
def MPJPE2(y_true, y_pred):
    yt= y_true.numpy().reshape((-1,32,2))
    yp= y_pred.numpy().reshape((-1,32,2))
    dist= np.zeros(10)
    for i in range(10):
        dist[i] = np.linalg.norm(yt[i] - yp[i])
    return np.mean(dist)

def PJPE2(y_true, y_pred):
    yt= y_true.numpy().reshape((10,64))
    yp= y_pred.numpy().reshape((10,64))
    dist= np.zeros(10)
    for i in range(10):
        dist[i] = np.linalg.norm(yt[i] - yp[i])
    return dist

In [18]:
# for layer in glogen_model.layers:
#     print(layer.output_shape)

In [19]:
# from keras import backend as K
# def compute_kernel(x, y):
#     # x=tf.expand_dims(x, axis=-1)
#     # y=tf.expand_dims(y, axis=-1)
#     x_size = K.shape(x)[0] #77144
#     y_size = K.shape(y)[0] #77144
#     dim = K.shape(x)[1] #10
#     #dim= 10,64
#     tiled_x = K.tile(K.reshape(x, [x_size, 1, dim]), [1, y_size, 1])
#     tiled_y = K.tile(K.reshape(y, [1, y_size, dim]), [x_size, 1, 1])
#     return K.exp(-K.mean(K.square(tiled_x - tiled_y), axis=2) / K.cast(dim, 'float32'))

# def compute_mmd(x, y):
#     x_kernel = compute_kernel(x, x)
#     y_kernel = compute_kernel(y, y)
#     xy_kernel = compute_kernel(x, y)
#     return K.mean(x_kernel) + K.mean(y_kernel) - 2 * K.mean(xy_kernel)
# #Trying MMD loss instead
history = run_experiment(epochs=10, lambda1=0.8, lambda2=0.2, metrics=[MPJPE2])

Epoch 1/10
618/618 [==============================] - 31s 38ms/step - loss: 77599352.0000 - MPJPE2: 3014.2136 - val_loss: 43005436.0000 - val_MPJPE2: 2268.2800
Epoch 2/10
618/618 [==============================] - 23s 38ms/step - loss: 26647932.0000 - MPJPE2: 1703.2275 - val_loss: 15255072.0000 - val_MPJPE2: 1316.8422
Epoch 3/10
618/618 [==============================] - 24s 38ms/step - loss: 10848242.0000 - MPJPE2: 1097.5651 - val_loss: 8374320.5000 - val_MPJPE2: 973.1194
Epoch 4/10
618/618 [==============================] - 23s 37ms/step - loss: 7496513.0000 - MPJPE2: 904.0046 - val_loss: 7389505.0000 - val_MPJPE2: 917.6710
Epoch 5/10
618/618 [==============================] - 26s 41ms/step - loss: 7085911.5000 - MPJPE2: 867.8441 - val_loss: 7337661.5000 - val_MPJPE2: 914.9301
Epoch 6/10
618/618 [==============================] - 23s 37ms/step - loss: 7046922.0000 - MPJPE2: 895.0511 - val_loss: 7291641.5000 - val_MPJPE2: 912.6095
Epoch 7/10
618/618 [==============================] - 